In [1]:
import torch
import torchvision

from utils.data_util import *
from utils.train_util import *
from torch.utils.data import DataLoader

net1 = torchvision.models.resnet18(weights=None, num_classes=10)
loss_func = torch.nn.CrossEntropyLoss()
num_device = torch.cuda.device_count()
device_list = [i for i in range(num_device)]
print(num_device, device_list)

net1 = torch.nn.DataParallel(net1.cuda(), device_ids=device_list, output_device=device_list[0])
loss_func = torch.nn.DataParallel(loss_func.cuda(), device_ids=device_list, output_device=device_list[0])

3 [0, 1, 2]


In [2]:
train_dataset, test_dataset, c, h, w = get_dataset('cifar10')
trainloader = DataLoader(
    dataset=train_dataset,
    batch_size=320,
    shuffle=True,
    pin_memory=True,
    num_workers=20)
testloader = DataLoader(
    dataset=test_dataset,
    batch_size=160,
    shuffle=True,
    pin_memory=True,
    num_workers=20)

Files already downloaded and verified


In [3]:
loss_func = torch.nn.CrossEntropyLoss().cuda()
for i in range(10):
    trained_model = deepcopy(net1).cuda()
    trained_model.train()
    optimizer = torch.optim.Adam(trained_model.parameters())
    loss_ = []
    for data, target in trainloader:
        optimizer.zero_grad()
        output = trained_model(data.cuda())
        loss = loss_func(output, target.cuda())
        loss.backward()
        optimizer.step()
        loss_.append(loss.item())
    print(i)

0
1
2
3
4
5
6
7


KeyboardInterrupt: 

In [ ]:
acc = eval_model(net1.eval(), testloader, 'cuda')
print(acc)

0.10009999573230743
